## US National Parks & Yelp! Reviews

In [1]:
# dependencies
import pandas as pd
from sqlalchemy import create_engine
import requests
import json
from pprint import pprint
import time 

# API and DB Credentials
# from keys import db_cred
from keys import gkey
from keys import connection_string



### Extract Park Data from JSON


In [2]:
json_file = "resources/data.json"
parks_df = pd.read_json(json_file)
parks_df.head()

,area,coordinates,date_established_readable,date_established_unix,description,image,nps_link,states,title,id,visitors,world_heritage_site
0,"{'acres': '49,057.36', 'square_km': '198.5'}","{'latitude': 44.35, 'longitude': -68.21}","February 26, 1919",-1604599200,Covering most of Mount Desert Island and other...,"{'url': 'acadia.jpg', 'attribution': 'PixelBay...",https://www.nps.gov/acad/index.htm,"[{'id': 'state_maine', 'title': 'Maine'}]",Acadia,park_acadia,"3,303,393",False
1,"{'acres': '8,256.67', 'square_km': '33.4'}","{'latitude': -14.25, 'longitude': -170.68}","October 31, 1988",594280800,The southernmost National Park is on three Sam...,"{'url': 'american-samoa.jpg', 'attribution': '...",https://www.nps.gov/npsa/index.htm,"[{'id': 'state_american-samoa', 'title': 'Amer...",American Samoa,park_american-samoa,"28,892",False
2,"{'acres': '76,678.98', 'square_km': '310.3'}","{'latitude': 38.68, 'longitude': -109.57}","November 12, 1971",58773600,"This site features more than 2,000 natural san...","{'url': 'arches.jpg', 'attribution': 'PixelBay...",https://www.nps.gov/arch/index.htm,"[{'id': 'state_utah', 'title': 'Utah'}]",Arches,park_arches,"1,585,718",False
3,"{'acres': '242,755.94', 'square_km': '982.4'}","{'latitude': 43.75, 'longitude': -102.5}","November 10, 1978",279525600,"The Badlands are a collection of buttes, pinna...","{'url': 'badlands.jpg', 'attribution': 'PixelB...",https://www.nps.gov/badl/index.htm,"[{'id': 'state_south-dakota', 'title': 'South ...",Badlands,park_badlands,"996,263",False
4,"{'acres': '801,163.21', 'square_km': '3,242.2'}","{'latitude': 29.25, 'longitude': -103.25}","June 12, 1944",-806439600,Named for the prominent bend in the Rio Grande...,"{'url': 'big-bend.jpg', 'attribution': 'PixelB...",https://www.nps.gov/bibe/index.htm,"[{'id': 'state_texas', 'title': 'Texas'}]",Big Bend,park_big-bend,"388,290",False


### Transform Park Data

In [28]:
natl_parks=parks_df.drop(columns=['date_established_unix','image', 'nps_link','states','area','coordinates'])
natl_parks.set_index('title',drop=True)
natl_parks.head()

,date_established_readable,description,title,id,visitors,world_heritage_site
0,"February 26, 1919",Covering most of Mount Desert Island and other...,Acadia,park_acadia,"3,303,393",False
1,"October 31, 1988",The southernmost National Park is on three Sam...,American Samoa,park_american-samoa,"28,892",False
2,"November 12, 1971","This site features more than 2,000 natural san...",Arches,park_arches,"1,585,718",False
3,"November 10, 1978","The Badlands are a collection of buttes, pinna...",Badlands,park_badlands,"996,263",False
4,"June 12, 1944",Named for the prominent bend in the Rio Grande...,Big Bend,park_big-bend,"388,290",False


In [4]:
# park_list = []

# for target_search in natl_parks['title']:
#     park_list.append(target_search)

# park_list

### Extract google places Reviews


In [5]:
# use Park DataFrame for names
# target_search = "acadia national park"


parks_dict = {}

for target_search in natl_parks['title']:
    params = {    
        "input": target_search + " national park",
        "inputtype": "textquery",
        "key": gkey
    }

    base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"

    response = requests.get(base_url, params=params)

    google_parks_data = response.json()
    park_id = google_parks_data['candidates'][0]['place_id']
    parks_dict[target_search] = park_id


In [6]:
parks_dict

{'Acadia': 'ChIJJSmiDrKjrkwRhFVV_A4i32I',
 'American Samoa': 'ChIJRx3UY-WEpnERNJZ__Rw4Hy8',
 'Arches': 'ChIJUaoNhhr2yoARlcQo0WnqQk8',
 'Badlands': 'ChIJIyAFGAyHfocR8Hfm2jFeaGk',
 'Big Bend': 'ChIJK-_lS3AX8oYRD4WDJ7x9RmQ',
 'Biscayne': 'ChIJK4Wkv7jb2YgRv7LTOSIRIRI',
 'Black Canyon of the Gunnison': 'ChIJ8Wo3SfxeP4cRvIuts5LWMT8',
 'Bryce Canyon': 'ChIJLevDAsZrNYcRBm2svvvY6Ws',
 'Canyonlands': 'ChIJqaYYRe7hR4cRquYCxalSpBU',
 'Capitol Reef': 'ChIJU6LnB_8ASocRB_9PSFPsO94',
 'Carlsbad Caverns': 'ChIJW9e4xBN544YRvbI7vfc91G4',
 'Channel Islands': 'ChIJFXm2LM5S6IARzDo6cliV5PU',
 'Congaree': 'ChIJBwa8iRQ1_4gRmiPbgxV26CE',
 'Crater Lake': 'ChIJOePlQAgXxlQRo-MvReHyK5A',
 'Cuyahoga Valley': 'ChIJdfUpIkjeMIgRquWefEq-yco',
 'Death Valley': 'ChIJR4qudndLx4ARVLDye3zwycw',
 'Denali': 'ChIJBRbQiP_zzVYRpTYZExU7tcY',
 'Dry Tortugas': 'ChIJP5KVUbD4zogR5scig326JKo',
 'Everglades': 'ChIJ2wVsejCo0IgRlzEvdlY-4A8',
 'Gates of the Arctic': 'ChIJbcXUoz4vKFERYkhcRDyuSp0',
 'Glacier': 'ChIJVVVVVRWQaFMR7F3FFSK8Fq8',


In [7]:
# google_parks_data = response.json()
# p_id = google_parks_data['candidates'][0]['place_id']
# print(p_id)

In [8]:
# possible .iterrows
# for key in parks_dict:
#     print(key, parks_dict[key])

In [23]:

park_title = []
park_name = []
park_rating = []
park_total_ratings = []

for key in parks_dict:

    if key != 'North Cascades':

        place_id = parks_dict[key]

        params2 = {    
            "place_id": place_id,
            "key": gkey
        }

        base_detail_url= "https://maps.googleapis.com/maps/api/place/details/json"

        try:

            response2 = requests.get(base_detail_url, params=params2)
            # time.sleep(10)
            park_details = response2.json()

            print(park_details["result"]["name"])

            park_title.append(key)
            park_name.append(park_details["result"]["name"])
            park_rating.append(park_details["result"]["rating"])
            park_total_ratings.append(park_details["result"]["user_ratings_total"])

        except Exception as e:
            print(e, key, park_details["result"]["name"])
            continue



Acadia National Park
National Park of American Samoa
'rating' American Samoa National Park of American Samoa
Arches National Park
Badlands National Park
Big Bend National Park
Biscayne National Park
Black Canyon of the Gunnison National Park
Bryce Canyon National Park
'rating' Bryce Canyon Bryce Canyon National Park
Canyonlands National Park
Capitol Reef National Park
Carlsbad Caverns National Park
Channel Islands National Park
Congaree National Park
Crater Lake National Park
Cuyahoga Valley National Park
Death Valley National Park
Denali National Park and Preserve
Dry Tortugas
'rating' Dry Tortugas Dry Tortugas
Everglades National Park
Gates of the Arctic National Park and Preserve
Glacier National Park
'rating' Glacier Glacier National Park
Glacier Bay Basin
Grand Canyon National Park
'rating' Grand Canyon Grand Canyon National Park
Grand Teton
Great Basin National Park
Great Sand Dunes National Park and Preserve
Great Smoky Mountains National Park
'rating' Great Smoky Mountains Grea

In [24]:
park_name

['Acadia National Park',
 'National Park of American Samoa',
 'Arches National Park',
 'Badlands National Park',
 'Big Bend National Park',
 'Biscayne National Park',
 'Black Canyon of the Gunnison National Park',
 'Bryce Canyon National Park',
 'Canyonlands National Park',
 'Capitol Reef National Park',
 'Carlsbad Caverns National Park',
 'Channel Islands National Park',
 'Congaree National Park',
 'Crater Lake National Park',
 'Cuyahoga Valley National Park',
 'Death Valley National Park',
 'Denali National Park and Preserve',
 'Dry Tortugas',
 'Everglades National Park',
 'Gates of the Arctic National Park and Preserve',
 'Glacier National Park',
 'Glacier Bay Basin',
 'Grand Canyon National Park',
 'Grand Teton',
 'Great Basin National Park',
 'Great Sand Dunes National Park and Preserve',
 'Great Smoky Mountains National Park',
 'Guadalupe Peak',
 'Haleakalā',
 'Hawaiʻi Volcanoes National Park',
 'Hot Springs National Park',
 'Isle Royale National Park',
 'Joshua Tree National Par

In [29]:
google_data = pd.DataFrame(zip(park_title, park_name, park_rating, park_total_ratings), columns=['title', 'name', 'rating', 'total_ratings'])

google_data.head()

,title,name,rating,total_ratings
0,Acadia,Acadia National Park,4.9,12080
1,American Samoa,National Park of American Samoa,4.8,19016
2,Arches,Arches National Park,4.9,10188
3,Badlands,Badlands National Park,4.8,3309
4,Big Bend,Big Bend National Park,4.7,1650


### Create Database Connection

In [5]:
connection=f'{connection_string}@localhost:5432/national_parks'
engine = create_engine(f'postgresql://{connection}')

ModuleNotFoundError: No module named 'psycopg2'

In [13]:
# yosemite_data=response2.json()
# pprint(yosemite_data)

In [14]:
# print(yosemite_data["result"]["name"])
# print(yosemite_data["result"]["rating"])
# print(yosemite_data["result"]["user_ratings_total"])

In [3]:
engine.table_names()

NameError: name 'engine' is not defined

In [16]:
natl_parks.to_sql(name='parks', con=engine, if_exists='append', index=False)

NameError: name 'engine' is not defined

In [17]:
google_data.to_sql(name='ratings', con=engine, if_exists='append', index=False)

NameError: name 'engine' is not defined

### Transform Yelp! Data

### Load DataFrames into Database